# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# total number of rows 
print(len(full_data_rows_list))

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# make a connection to a Cassandra instance your local machine (127.0.0.1)
from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1'])
    
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
## Query 1:  Give me the artist, song title and song's length in music app history that was heard during sessionId = 338, itemInSession = 4
## composite primary key (session_id, item_in_session)

query = "CREATE TABLE IF NOT EXISTS sessions "
query = query + "(session_id int, item_in_session int, artist_name text, song text, length float"
query = query + " ,PRIMARY KEY (session_id, item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)  

In [9]:
# Set up the CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO sessions (session_id, item_in_session, artist_name, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        
        ## Assigning which column elements should be mapped for each column in the INSERT statement
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## Add in the SELECT statement to verify the data was entered into the table
query = "select artist_name, song, length from sessions WHERE session_id=338 AND item_in_session=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
# use pretty table to display data in tabular form, including headings
t = PrettyTable(['Artist', 'Song', 'Length'])
for row in rows:
    t.add_row([row.artist_name, row.song, row.length])
print (t)

+-----------+---------------------------------+--------------------+
|   Artist  |               Song              |       Length       |
+-----------+---------------------------------+--------------------+
| Faithless | Music Matters (Mark Knight Dub) | 495.30731201171875 |
+-----------+---------------------------------+--------------------+


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [14]:
## Query 2: Retrieve name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
## composite primary key (user_id, session_id)
## clustering column "item_in_session" will determine sort order in a partition

query = "CREATE TABLE IF NOT EXISTS user_sessions "
query = query + "(user_id int, session_id int, artist_name text, song text, first_name text, last_name text, item_in_session int,"
query = query + "  PRIMARY KEY ((user_id, session_id), item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e) 

In [15]:
# Set up the CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_sessions (user_id, session_id, artist_name, song, first_name, last_name, item_in_session)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        ## Assigning which column elements should be mapped for each column in the INSERT statement
        session.execute(query, (int(line[10]), int(line[3]), line[0], line[9], line[1], line[4], int(line[8])))

In [16]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = "select artist_name, song, first_name, last_name from user_sessions WHERE user_id=10 AND session_id=182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
# use pretty table to display data in tabular form, including headings
t = PrettyTable(['Artist', 'Song', 'FirstName', 'LastName'])
for row in rows:
    t.add_row([row.artist_name, row.song, row.first_name, row.last_name])
print (t)

+--------+------+-----------+----------+
| Artist | Song | FirstName | LastName |
+--------+------+-----------+----------+
+--------+------+-----------+----------+


In [17]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
## composite primary key (song, user_id)
## if only song is used as primary key, when multiple users listen to same song this would overwrite and end up storing only last record

query = "CREATE TABLE IF NOT EXISTS song_listeners "
query = query + "(song text, user_id int, first_name text, last_name text,"
query = query + "  PRIMARY KEY (song, user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)              

In [18]:
# Set up the CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        ## Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_listeners (song, user_id, first_name, last_name)"
        query = query + " VALUES (%s, %s, %s, %s)"
        
        ## Assigning which column elements should be mapped for each column in the INSERT statement
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [19]:
## Add in the SELECT statement to verify the data was entered into the table
query = "select first_name, last_name from song_listeners WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
# use pretty table to display data in tabular form, including headings
t = PrettyTable(['FirstName', 'LastName'])
for row in rows:
    t.add_row([row.first_name, row.last_name])
print (t)

+------------+----------+
| FirstName  | LastName |
+------------+----------+
| Jacqueline |  Lynch   |
|   Tegan    |  Levine  |
|    Sara    | Johnson  |
+------------+----------+


### Drop the tables before closing out the sessions

In [20]:
## Drop the table before closing out the session
query = "DROP TABLE IF EXISTS sessions"
try:
    session.execute(query)
except Exception as e:
    print(e)  

In [21]:
## Drop the table before closing out the session
query = "DROP TABLE IF EXISTS user_sessions"
try:
    session.execute(query)
except Exception as e:
    print(e) 

In [22]:
## Drop the table before closing out the session
query = "DROP TABLE IF EXISTS song_listeners"
try:
    session.execute(query)
except Exception as e:
    print(e) 

### Close the session and cluster connection¶

In [23]:
session.shutdown()
cluster.shutdown()